In [1]:
import sys
sys.path.append('../bmpc_shared_scripts/prepare_dataset')

In [3]:
from dlomix.data.fragment_ion_intensity import FragmentIonIntensityDataset
from dlomix.constants import PTMS_ALPHABET
import tensorflow as tf
from pyarrow import parquet as pq
from dlomix.losses import masked_spectral_distance
from get_updated_alphabet import get_modification
from tqdm import tqdm

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# load a model
MODEL_DIR = '/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024_unmod_extended/'
RUN_NAME = '7ef3360f-2349-46c0-a905-01187d4899e2'
model = tf.keras.models.load_model(MODEL_DIR + RUN_NAME + '.keras')

2024-07-18 14:29:07.634554: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NOT_INITIALIZED: initialization error
2024-07-18 14:29:07.634624: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: minotaur.exbio.wzw.tum.de
2024-07-18 14:29:07.634655: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: minotaur.exbio.wzw.tum.de
2024-07-18 14:29:07.634846: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 550.90.7
2024-07-18 14:29:07.634932: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 550.90.7
2024-07-18 14:29:07.634948: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 550.90.7


In [6]:
small_parquet_path = '/cmnfs/proj/bmpc_dlomix/datasets/parquet/noptm_baseline_small_train.parquet'
ion_types = ['y', 'b']

In [7]:
# check if intensities column in parquet file
inference_only = True
col_names = pq.read_schema(small_parquet_path).names
if 'intensities_raw' in col_names:
    inference_only = False

In [8]:
# get all tokens present in the dataset
file = pq.ParquetFile(small_parquet_path)
dataset_tokens = set()
for batch in tqdm(file.iter_batches()):
    for cur_seq in batch['modified_sequence']:
        cur_mods = get_modification(str(cur_seq))
        dataset_tokens |= set(cur_mods)

1it [00:00,  9.38it/s]


In [9]:
# get the model alphabet and compare with the tokens from the dataset
# if new modifications are present -> need new embedding layer
model_tokens = set(model.alphabet.keys())
difference = dataset_tokens - model_tokens
if not difference:
    print('No tokens unknown to the model appear in the dataset!')
    new_alphabet = model.alphabet
else:
    print(f'These tokens appear in the dataset, but are not known to the model {difference}')
    print('A new embedding layer is necessary.')
    old_alphabet = model.alphabet
    new_alphabet = old_alphabet.update({k: i for i, k in enumerate(difference, start=len(model.alphabet) + 1)})

No tokens unknown to the model appear in the dataset!


In [10]:
# check for the ion types -> if ion types contain other than the b and y ions -> new output layer is necessary
number_of_ions = len(ion_types)
if any([ion_type in ['c', 'z', 'a', 'x'] for ion_type in ion_types]):
    if len(number_of_ions) == 2:
        print(f'New ion types detected, but only 2 ion types present. -> reinitialize the output layer')
    if len(number_of_ions) > 2:
        if 'y' in ion_types and 'b' in ion_types:
            print('New Ion types in addition to y and b ions detected -> new output layer, but can keep trained weights for y and b ions')
else:
    print('No new ion types detected. Output layer can stay the same.')

No new ion types detected. Output layer can stay the same.


In [26]:
ds = FragmentIonIntensityDataset(
    data_source=small_parquet_path,
    data_format='parquet',
    inference_only=inference_only,
    alphabet=new_alphabet,
    encoding_scheme='naive-mods',
    model_features=["precursor_charge_onehot", "collision_energy_aligned_normed", "method_nbr"]
)

Casting the dataset: 100%|██████████| 1598/1598 [00:00<00:00, 4888.46 examples/s]


In [27]:
ds.hf_dataset

DatasetDict({
    train: Dataset({
        features: ['modified_sequence', 'intensities_raw', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
        num_rows: 6390
    })
    val: Dataset({
        features: ['modified_sequence', 'intensities_raw', 'precursor_charge_onehot', 'collision_energy_aligned_normed', 'method_nbr', '_parsed_sequence', '_n_term_mods', '_c_term_mods'],
        num_rows: 1598
    })
})

### test the process_dataset function on different use cases

In [55]:
import sys
sys.path.append('../bmpc_shared_scripts/oktoberfest_interface')
from oktoberfest_interface import process_dataset

In [44]:
# different dataset paths
etd_dataset = '/cmnfs/proj/bmpc_dlomix/datasets/parquet/new_ion_types_ETD_support_edited.parquet'
inference_only_ds = 'test_inference_only.parquet'
single_ptm = '/cmnfs/data/proteomics/Prosit_PTMs/21PTMs/Kmod_Formyl.parquet'

In [45]:
# model path
model_path = '/cmnfs/proj/bmpc_dlomix/models/baseline_models/noptm_baseline_full_bs1024_unmod_extended/7ef3360f-2349-46c0-a905-01187d4899e2.keras'

In [56]:
ds, model = process_dataset(etd_dataset, model_path, ion_types=['z', 'c'])

/cmnfs/home/f.kapitza/dlomix/finn_notebooks/../bmpc_shared_scripts/oktoberfest_interface/oktoberfest_interface.py:69: UserWarning: 
                Number of ions is the same as the loaded model supports, but the ion types are different.
                The model probably needs to be refined to achieve a better performance on these new ion types.
                
  """


Start processing the dataset...


Casting the dataset: 100%|██████████| 1575/1575 [00:00<00:00, 2930.95 examples/s]

The available data splits are: train, val


dict_keys(['train', 'val'])

In [66]:
ds, model = process_dataset(inference_only_ds, 'unmod_ext', ion_types=['y', 'b'])

Start processing the dataset...


Generating train split: 7878 examples [00:00, 190705.20 examples/s]
/cmnfs/home/f.kapitza/dlomix/src/dlomix/data/dataset.py:366: UserWarning: 
                This is a inference only dataset! You can only make predictions with this dataset! Attempting to
                train a model with this dataset will result in an error!
                
  warnings.warn(
Casting the dataset: 100%|██████████| 7875/7875 [00:01<00:00, 6177.81 examples/s]

The available data splits are: inference


In [72]:
ds, model = process_dataset(single_ptm, 'unmod_ext', modifications=['K[UNIMOD:1]'])

/cmnfs/home/f.kapitza/dlomix/finn_notebooks/../bmpc_shared_scripts/oktoberfest_interface/oktoberfest_interface.py:58: UserWarning: 
            There are new tokens in the dataset, which are not supported by the loaded model.
            Either load a different model or transfer learning needs to be done.
            
  warnings.warn(


Start processing the dataset...


Casting the dataset: 100%|██████████| 1710/1710 [00:00<00:00, 3025.42 examples/s]

The available data splits are: train, val
